In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%capture
!pip3 install transformers
!pip3 install emoji
!pip3 install torchmetrics

In [ ]:
from PIL import Image
import requests
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration
import torch
import json
import numpy as np
import os
import pandas as pd
import emoji

In [ ]:
import torch
import clip
from PIL import Image

# Load the pre-trained CLIP model and the image
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-L/14@336px')
model.eval()
model = model.to(device)


def get_clip_score(image, text):

    # Preprocess the image and tokenize the text
    image_input = preprocess(image).unsqueeze(0)
    text_input = clip.tokenize([text])

    # Move the inputs to GPU if available
    image_input = image_input.to(device)
    text_input = text_input.to(device)

    # Generate embeddings for the image and text
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        text_features = model.encode_text(text_input)

    # Normalize the features
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)

    # Calculate the cosine similarity to get the CLIP score
    clip_score = torch.matmul(image_features, text_features.T).item()

    return clip_score*100

In [ ]:
THRESHOLD = 3.41
matches = 0
total = 0
NUM_IMAGES = 7

In [ ]:
root = "./PROP NOUN/"
df = pd.read_csv(root+"PROPNOUN_prompt_pairs_100.csv")

./PROP NOUN/


In [ ]:
for j in range(len(df)):
    input_text = df["input_prompt"][j]
    target_text = df["target_prompt"][j]

    dir_path = root+input_text.replace(" ","_").replace(".","_")+"___"+target_text.replace(" ","_").replace(".","_")+"/"
    sub_dir = os.listdir(dir_path)

    with open(dir_path + "/prompts.json", "r") as f:
        prompt = json.load(f)

    prompts = []
    for prm in prompt['adv_prompts']:
        if emoji.emojize(prm) == input_text:
            continue
        prompts.append(emoji.emojize(prm))

    total_avg_semSR = 0
    idx = 0
    success = 0
    avg_semSR = 0
    for dirs in sub_dir:
        if dirs.startswith("prompt_"):

            text_outputs1, text_outputs2, prm_outputs = [], [], []
            cnt = 0
            for images in os.listdir(os.path.join(dir_path, dirs)):
                if images.startswith("grid"):
                    continue
                imagelist = Image.open(os.path.join(dir_path, dirs)+"/"+images)

                inp_score = get_clip_score(imagelist, input_text)
                tar_score = get_clip_score(imagelist, target_text)
                prm_score = get_clip_score(imagelist, prompts[idx])

                #print(inp_score, " == ", tar_score)
                prm_outputs.append(prm_score)
                text_outputs1.append(inp_score)
                text_outputs2.append(tar_score)
                cnt = cnt + 1

            idx = idx + 1
            #print(cnt)
            scores = []
            semSR = 0
            for i in range(cnt):
                score = 0
                if text_outputs2[i] - text_outputs1[i] >= THRESHOLD:
                    score += 1
                else:
                    score -= 1
                try:
                    semSR += (prm_outputs[i] - text_outputs1[i])/(text_outputs2[i] - text_outputs1[i])
                except ZeroDivisionError:
                    semSR += 0

                scores.append(score)

            avg_semSR += semSR/cnt

            vote = (cnt//2)+1
            if scores.count(1) >= vote:
                success += 1

    try:
        total_avg_semSR = avg_semSR/idx
    except:
        total_avg_semSR = 0

    if success > 0:
        prompt["success"] = 1
    else:
        prompt["success"] = 0

    prompt["Avg SemSR"] = total_avg_semSR
    # print(j, " , ", success, " , ", total_avg_semSR)

    with open(dir_path + "/prompts.json", "w") as f:
        json.dump(prompt, f, indent=4)

In [ ]:
for_csv2 = {
    "input_text": [],
    "target_text": [],
    "success": [],
    "semSR":[]
}

for folder_name in os.listdir(root):
    folder_name = os.path.join(root, folder_name)
    if folder_name.endswith(".csv") or folder_name.endswith("checkpoints"):
        continue
    prompt = None
    with open(os.path.join(folder_name, 'prompts.json')) as f:
        prompt = json.load(f)

    for_csv2["input_text"].append(prompt["input_text"])
    for_csv2["target_text"].append(prompt["target_text"])
    for_csv2["success"].append(prompt["success"])
    for_csv2["semSR"].append(prompt["Avg SemSR"])

import pandas as pd

df = pd.DataFrame(for_csv2)

df.to_csv("propnoun_clip_result.csv", index=False)